## CSCE 676 :: Data Mining and Analysis :: Texas A&M University :: Spring 2026


# Weekly Homework 3: Graphs!


***Goals of this homework:***
Perform an analysis of a graph of your choice.


***Submission instructions:***

You should post your notebook to Canvas (look for the assignment there). Please name your submission **your-uin_hw3.ipynb**, so for example, my submission would be something like **555001234_hw3.ipynb**. Your notebook should be fully executed when you submit ... so run all the cells for us so we can see the output, then submit that.

***Grading philosophy:***

We are grading reasoning, judgment, and clarity, not just correctness. Show us that you understand the data, the constraints, and the limits of your conclusions.

***For each question, you need to respond with 2 cells:***
1. **[A Code Cell] Your Code:**
  - If code is not applicable for the question, you can skip this cell.
  - For tests: tests can be simple assertions or checks (e.g., using `assert` or `print` or small functions or visual inspection); formal testing frameworks are not required.
2. **[A Markdown Cell] Your Answer:** Write up your answers and explain them in complete sentences. Include any videos in this section as well; for videos, upload them to your TAMU Google Drive, and ensure they are set to be visible by the instruction team (set to: **anyone with a TAMU email can view**), then share the link to the video in the cell.

***At the end of each Section (A/B/C/...) include a cell for your resources:***

**[A Markdown Cell] Your Resources:** You need to cite 3 types of resources and note how they helped you: (1) Collaborators, (2) Web Sources (e.g. StackOverflow), and (3) AI Tools (you must also describe how you prompted, but we do not require any links to any specific chats). Specifically, use the following format as a template:
```
On my honor, I declare the following resources:
1. Collaborators:
- Reveille A.: Helped me understand that a df in pandas is a data structure kinda like a CSV.
- Sully A.: Helped me fix a bug with the vector addition of 2 columns.
- ...

2. Web Sources:
- https://stackoverflow.com/questions/46562479/python-pandas-data-frame-creation: how to create a pd df
- ...

3. AI Tools:
- ChatGPT: I gave it the homework .ipynb file and the ufo.csv, and told it to generate the code for the first question, but it did it with csv.reader(), so I re-prompted it to use pandas and that one was correct
- ...
```
***Why do we require this cell?*** This cell is important...

1. For academic integrity, you must give credit where credit is due.

2. We want you to pay attention to how you can successfully get help to move through problems! Is there someone you work with or an AI tool that helps you learn the material better? That's great! The point of engineering is to use your tools to solve hard problems, and part of graduate school is learning about how *you* learn and solve problems best.

***A reminder: you get out of it what you put into it.***
Do your best on these homeworks, show us your creativity, and ask for help when you need it -- good luck!

# A [72pts]. Step-by-Step Data Mining & Experimental Analysis on A Graph of Your Choice

**Rubric**

[18 pts] Strong/Professional: Correct and complete implementation of the task; Reasonable assumptions, stated or implied, and justified; Thoughtful handling of real-world data issues (missingness, noise, scale, duplicates, edge cases); Clear, concise explanations of what was done and why; Code is clean, readable, and well-structured, uses appropriate pandas, and would plausibly pass a professional code review; Tests meaningfully validate non-trivial behavior (not just "the code runs so it must be right").

[9 pts] Partial/Developing: Core task mostly completed but with gaps, weak assumptions, or minor mistakes; Reasoning is shallow or mostly descriptive; Code works but is messy, repetitive, or fragile; Tests are superficial, incomplete, or poorly motivated.

[0 pts] Minimal/Incorrect: Task is largely incorrect, missing, or misunderstands the goal; Little to no reasoning or justification; Code does not run or ignores constraints; No meaningful tests.


## Overview
In this homework, you will **choose one dataset you like** from the [SNAP datasets](https://snap.stanford.edu/data/index.html) collection from the **Social networks** section. You must choose a  **directed graph only**. In this section, you will perform a step‑by‑step data mining & experimental analysis. *Much of this section is self-directed, meaning you will need to make critical decisions about what tools you use and what you explore.*

Ideally, you should eventually turn in a coherent story: **What you tried → Why → What you found → So what? → Wait...Anything more?**. It's completely OKAY if you only get minor discoveries. But you should always document the whole learning and reasoning process. Grading will be based on the logic and coherence of your submitted notebook.

As a guide, for each step of the homework, you should briefly document:
- **Method choice & rationale.** Why this method? What do you expect?
- **Parameters.** E.g., `alpha=0.85` for PageRank; seed selection for PPR; thresholds for community extraction.
- **Results.** Tables/plots + **1–3 sentences** of interpretation.
- **Reflection.** Did results match your expectations? If not, why might that be?

Ideally, strong submissions should read like a *short research memo* rather than a raw dump of numbers.

## Environment Setup
You may use **Python 3.9+**. Other possible tools are:
- **Graph tools**: You are welcome to use existing tools that are optimised for graph learning. Among them, `networkx`is recommended. `graph-tool` and `igraph` are also good when dealing with larger graphs.
- **Data processing packages**: eg. `numpy`, `pandas`
- **Plotting**: `matplotlib`, `plotly`, `seaborn` etc.
- **other related tools** `scipy`, `scikit-learn`,  `pytorch` etc.
- You may choose other tools; just be sure to let us know.

> If your chosen dataset is very large, consider using `graph-tool` and `igraph`, or sampling/induced subgraphs to stay within reasonable time/memory limits.


In [16]:
# Install libraries as needed (uncomment if running on a clean environment)
# %pip install networkx pandas numpy matplotlib scipy scikit-learn
# %pip install python-louvain igraph

import os, io, gzip, zipfile, tarfile, sys, math, random
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import urllib.request
import shutil
import igraph as ig
import csv
# For reproducibility
random.seed(42)
np.random.seed(42)

print(nx.__version__)

3.2.1


# 1. Choose a Dataset
Pick **one** dataset from SNAP's **Directed networks** (e.g.`ego-Twitter`, `wiki-Vote` samples, etc.). Paste the **download URL** and a brief description of why you chose it.

- **Dataset name:** _e.g., soc-Slashdot0811_
- **URL:** _direct link to .txt/.gz_
- **Why this dataset?** _1–3 sentences on interest & expected properties_

> ⚠️ Make sure it's **directed**.

Add this cite to your citation cell:
> Jure Leskovec and Andrej Krevl. SNAP Datasets: Stanford Large Network Dataset Collection. http://snap.stanford.edu/data.

In [10]:
#NO ASSUMPTIONS MADE!
# === You may using the follow method to download datasets if you like ===
DATA_URL = "https://snap.stanford.edu/data/soc-Epinions1.txt.gz"  # Example; replace with your chosen dataset
LOCAL_PATH = "../data/raw_graph.txt.gz"

def download_dataset(url: str, to_path: str):
    import urllib.request
    print(f"Downloading from {url} ...")
    urllib.request.urlretrieve(url, to_path)
    size = os.path.getsize(to_path) / (1024*1024)
    print(f"Saved to {to_path} ({size:.2f} MB)")

# Uncomment to download when ready
download_dataset(DATA_URL, LOCAL_PATH)
#TESTS
import os
import re
import gzip
import pytest

def _write_dummy_gz(path: str, size_bytes: int = 1024) -> None:
    """Create a gz file of approximately size_bytes (uncompressed isn't important here)."""
    # We'll just write enough bytes into a gzip file to ensure os.path.getsize works.
    payload = b"a" * size_bytes
    with gzip.open(path, "wb") as f:
        f.write(payload)


import tempfile

def test_basic_download_mock():
    with tempfile.NamedTemporaryFile() as f:
        path = f.name

        def fake_urlretrieve(url, to_path):
            with open(to_path, "wb") as out:
                out.write(b"12345")

        urllib.request.urlretrieve = fake_urlretrieve

        download_dataset("http://fake.com", path)

        assert os.path.exists(path)
        assert os.path.getsize(path) > 0

test_basic_download_mock()
print("Test passed!")

Saved to ../data/raw_graph.txt.gz (1.55 MB)
Saved to /tmp/tmpy68jvuww (0.00 MB)
Test passed!


- **Dataset name:** Epinions Social Network
- **URL:** https://snap.stanford.edu/data/soc-Epinions1.txt.gz
- **Why this dataset?** I chose this dataset because it is reasonably sized (it can fit in main memory) and it is a directed graph. It will also be pretty easy to interpret the results of pagerank since each edge (a, b) means a trusts b. Thus, page rank in this graph simply implies that a given user is trustworthy: someone who is trusted by other people, who are themselves trusted, will have a high page rank.  

# 2. Load & Parse the Directed Graph
First choose a graph tool and write a few lines of why you choose it and how you are going to use it. If you are not using any tools, please also document that.

> SNAP directed edge lists are usually in the form `src\t dst` per line, with comments starting with `#`.

Implement a robust loader that:
- Skips comment lines (if there is any)
- Builds a **`networkx.DiGraph`** (if you choose networkx)
- (Optional) Restricts to the **largest weakly connected component (WCC)** for clarity

Write your loading graph code here:

In [20]:
def gunzip_file(file_path):
    """
    Decompress a .gz file into the same directory.

    Parameters
    ----------
    file_path : str
        Path to the .gz file

    Returns
    -------
    output_path : str
        Path to the decompressed file
    """

    # Make sure the file ends in .gz
    if not file_path.endswith(".gz"):
        raise ValueError("Input file must end with .gz")

    # Output path = remove .gz extension
    output_path = file_path[:-3]

    print(f"Decompressing {file_path} ...")

    # Open compressed file and write decompressed version
    with gzip.open(file_path, "rb") as f_in:
        with open(output_path, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

    print(f"Saved decompressed file to {output_path}")
    return output_path

RAW_GRAPH_PATH = gunzip_file(LOCAL_PATH)
    


def load_directed_graph(file_path, separator: str = " ") -> igraph.Graph:
    """Load a directed edge list into a igraph.Graph.
    Assumes lines like: u<sep>v, with comment lines starting by `comment`.
    If sep is None, split on whitespace.
    """
    
    """
    Notably, the igraph library has a lot of functionality, however reading a graph from a csv is not apart of that functionality.
    igraph.Graph constructor: __init__(n=0, edges=None, directed=False, graph_attrs=None, vertex_attrs=None, edge_attrs=None)
    igraph.Edge constructor: 
    """
    node2id = {}
    edges = []
    def get_id(label: str) -> int:
        if label not in node2id:
            node2id[label] = len(node2id)
        return node2id[label]

    with open(file_path, "rt") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue

            if separator is None:
                parts = line.split()
            else:
                parts = line.split(separator)

            # Some datasets may have extra columns; only take first two
            if len(parts) < 2:
                continue

            u, v = parts[0], parts[1]
            edges.append((get_id(u), get_id(v)))

    graph = igraph.Graph(n=len(node2id), edges=edges, directed=True)
    # Store original labels so you can map back later
    graph.vs["name"] = [None] * len(node2id)
    for label, idx in node2id.items():
        graph.vs[idx]["name"] = label

    return graph

raw_epinion_graph = load_directed_graph(RAW_GRAPH_PATH, separator="\t")
print("Loaded the raw epinion graph which has ", raw_epinion_graph.ecount(), " edges and ", raw_epinion_graph.vcount(), " vertices")
#TESTS
def test_gunzip():
    import tempfile

    # Create dummy gz file
    tmp_dir = tempfile.mkdtemp()
    gz_file = os.path.join(tmp_dir, "test.txt.gz")

    with gzip.open(gz_file, "wb") as f:
        f.write(b"hello world")

    # Run gunzip
    out_file = gunzip_file(gz_file)

    # Check contents
    with open(out_file, "rb") as f:
        assert f.read() == b"hello world"

    print("gunzip_file test passed!")


def test_load_directed_graph_basic():
    # Create a temporary edge list file
    content = """# This is a comment
    1 2
    2 3
    
    # Another comment
    3 1
    """

    with tempfile.NamedTemporaryFile(mode="w+", delete=False) as f:
        f.write(content)
        path = f.name

    # Load graph
    G = load_directed_graph(path)

    # Assertions
    assert G.is_directed(), "Graph should be directed"
    assert G.vcount() == 3, f"Expected 3 vertices, got {G.vcount()}"
    assert G.ecount() == 3, f"Expected 3 edges, got {G.ecount()}"

    # Check vertex names exist
    assert set(G.vs["name"]) == {"1", "2", "3"}

    print("✅ test_load_directed_graph_basic passed!")

def test_load_directed_graph_csv_separator():
    content = """A,B
B,C
C,A
"""

    with tempfile.NamedTemporaryFile(mode="w+", delete=False) as f:
        f.write(content)
        path = f.name

    # Load graph with comma separator
    G = load_directed_graph(path, separator=",")

    assert G.is_directed()
    assert G.vcount() == 3
    assert G.ecount() == 3

    assert set(G.vs["name"]) == {"A", "B", "C"}

    print("✅ test_load_directed_graph_csv_separator passed!")

test_load_directed_graph_csv_separator()
test_load_directed_graph_basic()
test_gunzip()

Decompressing ../data/raw_graph.txt.gz ...
Saved decompressed file to ../data/raw_graph.txt
Loaded the raw epinion graph which has  508837  edges and  75879  vertices
✅ test_load_directed_graph_csv_separator passed!
✅ test_load_directed_graph_basic passed!
Decompressing /tmp/tmpfu48m6z4/test.txt.gz ...
Saved decompressed file to /tmp/tmpfu48m6z4/test.txt
gunzip_file test passed!


I am choosing to use the igraph tool for graphs because it has some very useful built in analysis functions such as betweeness(), closeness(), and even pagerank(). I'm going to use this tool for analysis as well as visualization since the library also provides visualization functions.

# 3. First Look: Basic Structural Statistics
Compute and report at least:
- `|V|` (nodes), `|E|` (edges)
- **Average in/out degree**, **degree distributions** (plot)
- **#SCCs**, size of **largest SCC** and **largest WCC**
- *(Optional): **Density**, **reciprocity***

Add a few sentences interpreting what these numbers suggest about your network.


In [26]:
'''sample codes'''
import igraph as ig
import pandas as pd

def basic_stats(graph: ig.Graph) -> pd.DataFrame:
    """
    Compute basic directed-graph statistics and return them
    as a one-row pandas DataFrame.

    Stats included:
      - number of vertices, number of edges
      - number of SCCs, size of largest SCC
      - number of WCCs, size of largest WCC
      - density
      - reciprocity
    """

    # --- Basic size ---
    n = graph.vcount()
    m = graph.ecount()

    # --- Connected components ---
    scc = graph.components(mode="STRONG")
    wcc = graph.components(mode="WEAK")

    num_scc = len(scc)
    largest_scc = max((len(c) for c in scc), default=0)

    num_wcc = len(wcc)
    largest_wcc = max((len(c) for c in wcc), default=0)

    # --- Density ---
    density = graph.density(loops=False)

    # --- Reciprocity ---
    reciprocity = graph.reciprocity()

    # --- Return as DataFrame ---
    stats = {
        "num_vertices": n,
        "num_edges": m,
        "num_scc": num_scc,
        "largest_scc": largest_scc,
        "num_wcc": num_wcc,
        "largest_wcc": largest_wcc,
        "density": density,
        "reciprocity": reciprocity,
    }

    return pd.DataFrame([stats])

    
    

# Example after loading:
epinion_stats = basic_stats(raw_epinion_graph)
print("EPINION STATS")
display(epinion_stats)


#TESTS
def test_basic_stats():
    # Create a small directed graph
    # 0 -> 1 -> 2 -> 0 forms one SCC of size 3
    # 3 -> 4 is a separate weak component
    edges = [(0, 1), (1, 2), (2, 0), (3, 4)]
    G = ig.Graph(edges=edges, directed=True)

    # Run basic stats
    df = basic_stats(G)

    # Assertions
    assert df["num_vertices"][0] == 5
    assert df["num_edges"][0] == 4

    assert df["num_scc"][0] == 3   # {0,1,2}, {3}, {4}
    assert df["largest_scc"][0] == 3

    assert df["num_wcc"][0] == 2   # {0,1,2} and {3,4}
    assert df["largest_wcc"][0] == 3

    print("✅ test_basic_stats passed!")

test_basic_stats()

EPINION STATS


,num_vertices,num_edges,num_scc,largest_scc,num_wcc,largest_wcc,density,reciprocity
0,75879,508837,42176,32223,2,75877,0.000088,0.405226


✅ test_basic_stats passed!


From the above we see that we have a relatively sparse graph since the total number of edges is much less than the total possible number of edges given the number of vertices.  
Namely, there are $$75879 \cdot 75878 \approx 5.7 \cdot 10^9$$ possible edges however there are only $$5 \cdot 10^5$$ actual edges in this graph.  


# 4. Quick Visualization (Exploratory)
Produce a small subgraph visualization to build intuition (e.g., induced subgraph of top-`k` PageRank nodes or a random 500-node sample). For large graphs, you **don't have to** try to plot everything.

- Annotate what you observe (hubs? communities? sources/sinks?)


In [ ]:
def visualize_subgraph(G):



# 5.

Prepare a concise research brief summarizing your data mining results and their significance.

Your write-up must include:

1. Results
- Clearly and precisely report your empirical findings.
- Include relevant quantitative results (e.g., metrics, comparisons, trends, error analysis).
- Figures or tables may be used if they improve clarity, but they must be referenced and interpreted in the text.
- You can use markdown formatting (bold, italics, headings, etc.) to help you communicate your findings.
- Do not focus on implementation details unless they are necessary to understand the results.

2. Significance and Interpretation
- Explain why the results matter in a data mining context.
- Discuss what the findings imply about the data, the model(s), or the assumptions made.
- Address limitations, tradeoffs, or unexpected outcomes where relevant.

Clearly state what new insight is gained from your results.
Aim for 5-7 paragraphs in length.

```
On my honor, I declare the following resources:
1. Collaborators:
- 

2. Web Sources:
- Jure Leskovec and Andrej Krevl. SNAP Datasets: Stanford Large Network Dataset Collection. http://snap.stanford.edu/data. (used for searching for relevant datasets)
- https://snap.stanford.edu/data/soc-Epinions1.txt.gz - dataset used for analysis
- https://python.igraph.org/en/stable/ - documentation for igraph, used to find out how to use igraph (igraph API docs)

3. AI Tools:
- ChatGPT: Wrote the test cases for part 1, the generated code worked, i did not have to change it. 
- ChatGPT: Wrote the test cases for part 2, the generated code worked, and i did not have to change it. Also wrote the gunzip function, for which it also provided functional code. Also generated the vast majority of the load_directed_graph function, I just provided it with which parameters the function should accept and a high level overview of what the function should do.
- ChatGPT: Wrote the test cases for part 3, the generated code worked, and i did not have to change it. For the basic stats function, I just gave it the parameters and expected behavior and it filled in the rest, didn't have to modify the code.

```

# B [24pts]. Interview Questions

We now pretend this is a real job interview. Here's some guidance on how to answer these questions:

1. Briefly restate the question and state any assumptions you are making.

2. Explain your reasoning out loud, focusing on tradeoffs, limitations, and constraints.

3. As a principle, keep your answers as short and clear as they can be (while still answering the question).

4. Write/speak in a conversational but professional tone (avoid being overly formal). For speaking: speak at a reasonable pace and volume, speak clearly, pause when you need to, and practice making "eye contact" with the camera. Keep a confident, positive, and professional tone. *For additional coaching and practice, the University Writing Center provides individual appointments: https://writingcenter.tamu.edu/make-an-appointment.*

There may not be a single correct answer. We are grading whether your reasoning is reasonable and aware of limitations.


**Rubric**

[8pt] Clear understanding of the question; reasonable assumptions; thoughtful reasoning that acknowledges tradeoffs and limitations; clear, concise communication in a conversational but professional tone (for speaking: clear pace, volume, and articulation).

[4pt] Basic understanding but shallow reasoning or unclear assumptions; communication is somewhat unclear, overly verbose, or overly informal/formal.

[0pt] Minimal, unclear, or incorrect response; poor communication or unprofessional tone.

# 1.
Many real systems can be represented as graphs in multiple ways. How would you decide what the nodes and edges should represent in a given domain, and what kinds of errors can arise from a poor abstraction?

# 2.
Discuss how missing edges, spurious edges, or sampling bias affect centrality-based conclusions. Which measures are most fragile?

# 3.
As a video (reminder to keep it brief, 2 minutes max): So, I see you did a graph analysis (referring to this homework). That's cool -- can you walk me through what you did?

# C [4pts]. What new questions do you have?
We want you to think bigger! Tell us what questions and curiosity this homework brings up for you.

**Rubric**

[4pt] Complete, thoughtful response.

[2pt] Partial response.

[0pt] Minimal response.

# 1.
What new questions do you have about association rule mining (in general) after this homework? Or, what topics are you curious about now? List at least 3.